Подключаем библиотеки

In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

Загрузим данные в pandas dataframe:

In [3]:
data = pd.read_csv("/Users/Hun/Vk_csv_full_lem_CORRECTED.csv",
                   encoding="CP1251",
                   sep=";",
                   header=None,
                   parse_dates=[4],
                   usecols=range(9),
                   names=["text", "author_id", "wall_id", "post_comment_id", "date", "name", "surname", "city", "region"])

data.head


<bound method NDFrame.head of                                                       text  author_id  \
0                                             general lee    49154141   
1                                                  shelby    49154141   
2                                            ford mustang    49154141   
3                                              murcielago    49154141   
4                                        суровый водитель    49154141   
5                                                  shelby    49154141   
6                                        dodge challenger    49154141   
7                            мотоцикл старое наручный час    49154141   
8                               camaro camaro convertible    49154141   
9                                           dodge charger    49154141   
10                                   картин цветной скотч    49154141   
11                                  low ride buick rivera    49154141   
12                   

Посчитаем количество слов в тексте:

In [5]:
splitted_texts = data.text.str.split()
texts_length = [len(text) for text in splitted_texts]
data["text_length"] = pd.Series(texts_length)

Уберём из ID стены слово wall:

In [44]:
data["wall_id"] = data["wall_id"].apply(lambda text: int(text.split()[-1]))
data.head()

Сохраним (или загрузим) данные:

In [3]:
#data.to_pickle('hse_thesis1.pickle')
data = pd.read_pickle('hse_thesis1.pickle')

FileNotFoundError: [Errno 2] No such file or directory: 'hse_thesis1.pickle'

Статистика по данным

In [15]:
data.describe()

,author_id,wall_id,text_length
count,5.392586e+06,5.392586e+06,5392586.000000
mean,1.169800e+08,1.229665e+08,15.811299
std,8.157409e+07,7.888935e+07,32.214321
min,0.000000e+00,9.293000e+03,1.000000
25%,3.918839e+07,5.147288e+07,3.000000
50%,1.257181e+08,1.343828e+08,6.000000
75%,1.768054e+08,1.806884e+08,11.000000
max,3.280245e+08,2.944905e+08,6741.000000


In [26]:
data.isnull().sum()

text                    0
author_id               0
wall_id                 0
post_comment_id         0
date                    0
name                48759
surname             49346
city               411426
region             421852
text_length             0
dtype: int64

In [27]:
data.replace(np.nan, "missing_data", inplace=True)

Создадим агрегированную таблицу по id пользователя и году, учитывая только записи, которые пользователи оставляли на своей стене:

In [28]:
aggregations = {
    "region": "first"
}
aggregations_for_year = {
    "text": "count", # количество текстов
    "text_length": "sum" # количество токенов
}


data_full_own = data[data.author_id == data.wall_id].groupby('author_id').agg(aggregations)
data_full_own

for year in range(2007, 2016):
    data_year_own = data[(data.date.dt.year == year) & (data.author_id == data.wall_id)].groupby('author_id').agg(aggregations_for_year).rename(columns=lambda x: x + "_ownwall_" + str(year))
    data_full_own = pd.concat([data_full_own, data_year_own], axis=1)
#184561440
data_full_own

,region,text_length_ownwall_2007,text_ownwall_2007,text_length_ownwall_2008,text_ownwall_2008,text_length_ownwall_2009,text_ownwall_2009,text_length_ownwall_2010,text_ownwall_2010,text_length_ownwall_2011,text_ownwall_2011,text_length_ownwall_2012,text_ownwall_2012,text_length_ownwall_2013,text_ownwall_2013,text_length_ownwall_2014,text_ownwall_2014,text_length_ownwall_2015,text_ownwall_2015
author_id,,,,,,,,,,,,,,,,,,,
9293,Сахалинская область,13,4,98,25,55,18,101,28,821,40,1322,42,6225,105,2730,34,67,3
73221,Архангельская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1275,40,204,18,21,2,2,2
73462,Тюменская область,2,1,70,21,423,87,340,79,331,24,570,15,2638,40,1846,24,1413,17
95487,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1
96309,missing_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,3,16,4,11,2
108069,Тульская область,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,395,9,NaN,NaN,57,1,NaN,NaN
118755,Иркутская область,3,2,7,1,1,1,NaN,NaN,2,1,9,3,1428,55,3055,130,180,20
133576,Санкт-Петербург,291,68,46,16,18,9,NaN,NaN,NaN,NaN,29,1,NaN,NaN,3,2,67,3
165626,Краснодарский край,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2


Нужно посчитать статистику для всех постов со стен пользователей

In [29]:
data_full_wall = data[["wall_id"]].drop_duplicates("wall_id").set_index("wall_id")

for year in range(2007, 2016):
    data_year_wall = data[data.date.dt.year == year].groupby('wall_id').agg(aggregations_for_year).rename(columns=lambda x: x + "_" + str(year))
    data_full_wall = pd.concat([data_full_wall, data_year_wall], axis=1)
data_full_wall

,text_length_2007,text_2007,text_length_2008,text_2008,text_length_2009,text_2009,text_length_2010,text_2010,text_length_2011,text_2011,text_length_2012,text_2012,text_length_2013,text_2013,text_length_2014,text_2014,text_length_2015,text_2015
wall_id,,,,,,,,,,,,,,,,,,
9293,204,36,326,76,366,77,297,59,1058,62,1418,64,6477,139,2946,67,157,22
73221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1275,40,204,18,21,2,2,2
73462,212,21,1712,356,3418,565,2919,519,1647,222,1183,120,3026,111,1979,52,1427,20
80975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,6,22,6
96309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,3,16,4,11,2
108069,16,1,12,2,9,1,8,4,107,15,447,18,144,22,140,15,33,9
118755,12,7,76,21,38,12,171,22,40,8,35,11,1506,71,3338,152,181,21
133576,1411,293,366,91,318,63,86,22,49,13,114,17,20,6,15,5,67,3


Сливаем и сохраняем

In [30]:
final_data = pd.concat([data_full_own, data_full_wall], axis=1)
final_data.to_csv("final_data.csv")
final_data

,region,text_length_ownwall_2007,text_ownwall_2007,text_length_ownwall_2008,text_ownwall_2008,text_length_ownwall_2009,text_ownwall_2009,text_length_ownwall_2010,text_ownwall_2010,text_length_ownwall_2011,...,text_length_2011,text_2011,text_length_2012,text_2012,text_length_2013,text_2013,text_length_2014,text_2014,text_length_2015,text_2015
9293,Сахалинская область,13,4,98,25,55,18,101,28,821,...,1058,62,1418,64,6477,139,2946,67,157,22
73221,Архангельская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1275,40,204,18,21,2,2,2
73462,Тюменская область,2,1,70,21,423,87,340,79,331,...,1647,222,1183,120,3026,111,1979,52,1427,20
80975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95487,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18,6,22,6
96309,missing_data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24,3,16,4,11,2
108069,Тульская область,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,...,107,15,447,18,144,22,140,15,33,9
118755,Иркутская область,3,2,7,1,1,1,NaN,NaN,2,...,40,8,35,11,1506,71,3338,152,181,21
133576,Санкт-Петербург,291,68,46,16,18,9,NaN,NaN,NaN,...,49,13,114,17,20,6,15,5,67,3
137775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# НИЖЕ НЕ СМОТРЕТЬ. ТЕСТЫ, ЭКСПЕРИМЕНТЫ

In [1]:
own_posts_data = data[data.author_id == data.wall_id]
grouped = own_posts_data.groupby([own_posts_data.author_id, own_posts_data.date.dt.year])
own_posts_grouped = grouped.agg({"text_length": np.sum,
                  "text": np.size,
                  "region": lambda x: x.iloc[0]
                 })
own_posts_grouped

NameError: name 'data' is not defined

Плоская группировка:

In [90]:
groupby_author = data.groupby([data.wall_id, data.date.dt.year])

data_grouped = groupby_author.agg({"text_length": np.sum,
                  "text": np.size,
                  "region": lambda x: x.iloc[0],
            })
data_grouped.to_csv("posts_data.csv")
data_grouped

text                           region  text_length
wall_id   date                                                    
9293      2007    36              Сахалинская область          204
          2008    76                              NaN          326
          2009    77              Сахалинская область          366
          2010    59              Сахалинская область          297
          2011    62              Сахалинская область         1058
          2012    64              Сахалинская область         1418
          2013   139                  Санкт-Петербург         6477
          2014    67                  Санкт-Петербург         2946
          2015    22              Сахалинская область          157
73221     2012    40            Архангельская область         1275
          2013    18            Архангельская область          204
          2014     2            Архангельская область           21
          2015     2            Архангельская область            2
73462     2007    21                           Москва          212
          2008   356                Тюменская область         1712
          2009   565                Тюменская область         3418
          2010   519                Тюменская область         2919
          2011   222                Тюменская область         1647
          2012   120              Гродненская область         1183
          2013   111              Гродненская область         3026
          2014    52              Гродненская область         1979
          2015    20                Тюменская область         1427
80975     2007     1                  Санкт-Петербург            1
95487     2014     6                  Санкт-Петербург           18
          2015     6                  Санкт-Петербург           22
96309     2013     3                              NaN           24
          2014     4                              NaN           16
          2015     2                              NaN           11
108069    2007     1                           Москва           16
          2008     2                              NaN           12
...              ...                              ...          ...
294218015 2015    11               Пензенская область           46
294236410 2015     4                     Крым область           34
294247159 2015    30                 Тульская область          292
294260055 2015    31                          Хакасия          859
294261725 2015    18             Оренбургская область           84
294276488 2015    12              Саратовская область          155
294277014 2015     4            Волгоградская область           18
294282595 2015    12             Новгородская область          111
294288449 2015     2              Ярославская область           10
294299934 2015     3                              NaN           20
294305870 2015    10              Сахалинская область          292
294316019 2015     1             Владимирская область            1
294324471 2015    43                              NaN          733
294328742 2015     2                Красноярский край           13
294343414 2015    13                        Чеченская           62
294362095 2015    19              Вологодская область           82
294370634 2015    12              Черновицкая область           70
294383491 2015     9                          Карелия           39
294404877 2015     1  Севастопольский городской совет           40
294408554 2015     5              Кемеровская область           48
294415549 2015     1                Иркутская область            1
294422649 2015     2            Архангельская область           13
294432040 2015     1                        Татарстан            1
294433188 2015     1                  Санкт-Петербург            3
294434630 2015    24            Новосибирская область          116
294435017 2015    39                 Хабаровский край          132
294438777 2015     2                          Бурятия          

In [ ]:
data.groupby([data.author_id, data.date.dt.year]).count().unstack()